### 유형별 피처 분리

In [ ]:
CATBOOST_VERSION = 0.97
NFOLDS = 10
SEED = 0
NCOMP = 50
P = 0.05

In [ ]:
X_train = pd.read_csv('X_train_Kyh.csv', encoding='cp949')
y_train = pd.read_csv('y_train_Kyh.csv', encoding='cp949')

X_test = pd.read_csv('X_test_Kyh.csv', encoding='cp949')

In [ ]:
X_train.dtypes

In [ ]:
numeric_features = ['근무경력','대학성적','근무경력^2','NaN_count','NotNull_count','출신대학*대학성적']
categorical_features = ['직종','세부직종','출신대학','대학전공','어학시험','자격증','근무경력_세분화','마지막근무형태',
                       '근무지역_a','근무지역_b','근무지역_c','해외근무지역','단과대','대학전공_and_경력세분화','ID',
                       '근무경력_y', '근무경력_m', '근무지역_소분류','근무희망형태','어학시험언어','상위어학시험','new_근무지역',
                       '근무지역_경험횟수','세부직종_연봉','숙련도','신입경력','소프트웨어대+공대','대학+전공',
                       '근무형태+직종']
            
binary_features = ['직무태그', '근무지역','근무형태']

In [ ]:
X_train = X_train[numeric_features+categorical_features+binary_features]  # 순서 주의!!!
X_test = X_test[numeric_features+categorical_features+binary_features]

# CatBoost의 cat_features 파라미터에 지정할 범주형 피처 위치
cat_index = [list(X_train.columns).index(c) for c in categorical_features]

### 전처리 파이프라인 구축

In [ ]:
# 상하한값 제한을 통한 결측값 처리 함수: FunctionTransformer를 통해 호출
def remove_outlier(X, q=0.05):  
    df = pd.DataFrame(X)
    return df.apply(lambda x: x.clip(x.quantile(q), x.quantile(1-q)), axis=0).values

# 회귀분석의 계수검정을 이용한 피처선택 전처리기 클래스
class MyFeatureSelector(TransformerMixin, BaseEstimator):
    # 전처리기 생성 즉, MyFeatureSelector() 호출시 실행
    def __init__(self, p=0.01):
        self.p = p

    # 전처리기의 fit() 호출시 실행
    def fit(self, X, y=None):
        X = sm.add_constant(X)
        results = sm.OLS(y, X).fit()
        self.cols = list(results.pvalues[1:] <= self.p)
        return self
    
    # 전처리기의 transform() 호출시 실행
    def transform(self, X):
        return X[:,self.cols].astype(np.int64)        
    
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("outlier", FunctionTransformer(remove_outlier, kw_args={'q':0.05})),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")), 
        ("encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=99999, dtype=np.object)),
    ]
)

binary_transformer = Pipeline(
    steps=[
        ("impuer", FunctionTransformer(lambda x: x.fillna('없음'))),      
        ("corpus", FunctionTransformer(lambda x: x.str.replace('·',',').str.split(',').str.join(" "))),
        ("BoW", CountVectorizer()),
        ("dense", FunctionTransformer(lambda x: x.toarray().astype(int), accept_sparse=True)),
    ]
)

column_transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("bin1", make_pipeline(binary_transformer, TruncatedSVD(n_components=NCOMP,random_state=SEED)), binary_features[0]),
        ("bin2", make_pipeline(binary_transformer, MyFeatureSelector(p=P)), binary_features[1]),
        ("bin3", make_pipeline(binary_transformer, MyFeatureSelector(p=P)), binary_features[2]),
    ]
)

preprocessor = Pipeline(
    steps=[
        ("column", column_transformer), 
    ]
)

set_config(display="diagram")
preprocessor

In [ ]:
X_train = preprocessor.fit_transform(X_train, y_train)
X_test = preprocessor.transform(X_test)

### 모형생성

In [ ]:
%%time

# 최적화된 하이퍼파라미터로 OOF를 수행하여 최종 CatBoost 모형 생성:
# No tuning => tuning한 모델에 비해 성능이 떨어지지 않음

#sscv = ShuffleSplit(test_size=.3334, n_splits=5, random_state=0)
models = cross_validate(CatBoostRegressor(cat_features=cat_index, verbose=False, random_state=SEED),
                        X_train, y_train, 
                        cv=NFOLDS, 
                        scoring='neg_mean_squared_error', 
                        return_estimator=True)
oof_pred = np.array([m.predict(X_test) for m in models['estimator']]).mean(axis=0)

scores = models['test_score']
print("\nCatBoost CV scores: ", np.sqrt(-1*scores))
print("CatBoost CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))

In [ ]:
# CatBoost CV mean = 829.83 with std = 378.38

In [ ]:
# submission 화일 생성
filename = f'catboost_{CATBOOST_VERSION}_{np.sqrt(-1*scores.mean()):.2f}.csv'
pd.DataFrame({'ID':test_id, 'Salary':oof_pred}).to_csv(filename, index=False)

In [ ]:
pd.set_option('display.max_columns',None)